**Корректность проверена на Python 3.6:**
+ gensim 3.4.0

# Пример использования библиотеки gensim для тематического моделирования

Такая полезная теорема Байеса! :)

![comic1](http://imgs.xkcd.com/comics/seashell.png)

В `gensim` реализована модель LDA (Latent Dirichlet Allocation), обучение которой основано на эволюционном байесовском выводе. LDA, действительно, очень популярны, и часто их используют просто как «чёрный ящик», подают матрице частоты слов и получают на выходе построенные матрицы.

Мы импортируем компоненты corpora и models из модуля gensim. Первый поможет нам импортировать данные, а второй — построить непосредственно модель. 

In [1]:
from gensim import corpora, models

C:\Users\mi.terentev\.conda\envs\Orange\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Мы возьмём dataset, состоящий из текстовых описаний к комиксам [xkcd](https://xkcd.com/).

Данные у нас представлены в формате UCI Bag of Words. Это популярный формат, который позволяет максимально сжато представить разреженную матрицу частот слов. 

Итак, давайте импортируем эти файлы в нашу модель. Для этого мы создадим объект класса UciCorpus и укажем путь к двум нашим файлам. Также gensim требует, чтобы мы в отдельную переменную обязательно сохранили словарь. Это можно сделать с помощью метода create_dictionary у объекта data, который мы получили в первой строке. Теперь мы готовы к обучению модели. 

In [2]:
# Импортируем данные в формте UCI Bag of Words
data = corpora.UciCorpus("docword.xkcd.txt", "vocab.xkcd.txt")
dictionary = data.create_dictionary()

Первый параметр — это corpus, у нас это, он хранится в переменной data, мы её и указываем. Второй параметр, который, конечно, необязателен, но крайне желательно его указать, это id2word. Это, собственно, отображение индексов слов непосредственно в слова. У нас это как раз тот dictionary, который мы создали во второй строке предыдущей ячейки. Далее идут некоторые параметры, которые влияют на обучение модели. Если у вас коллекция небольшая, как в нашем случае, то можно сделать много проходов по коллекции, и тогда модель получится наиболее адекватной. Тогда нужно указать параметр passes большим, например 20. Если же у вас коллекция действительно большая, например, как «википедия», то делать по ней двадцать проходов будет очень долго. Тогда можно указывать один или два прохода, но нужно обязательно подобрать аккуратно другие параметры, влияющие на сходимость алгоритмов. Особенно это параметры decay и offset, но об этом мы не будем подробно говорить в нашей демонстрации. Кроме того, полезно указывать параметр distributed, например, в true, тогда вы сможете обучать модель сразу в нескольких параллельных процессах. Ещё два важных параметра модели — это alpha и eta. Они создают априорные параметры распределения Дирихле, точнее, параметры априорного распределения Дирихле для моделей. 

In [3]:
# обучение модели
%time ldamodel = models.ldamodel.LdaModel(data, id2word=dictionary, num_topics=5, passes=20, alpha=1.25, eta=1.25)

Wall time: 1min 1s


In [4]:
# Сохранение модели
ldamodel.save("ldamodel_xkcd")

In [5]:
# Загрузка модели
ldamodel = models.ldamodel.LdaModel.load("ldamodel_xkcd")

Мы видим, что в первых двух темах много имён, и, честно сказать, не очень понятно, о чём эти темы. Третья тема, она посвящена комиксам в таком стиле: там любят рисовать график, и этот график как-то связан со временем, и идёт какой-то сопровождающий текст. Потом говорят слова: «рисунок», «метка», «линия», «год», «время». Третья тема — это тема для комиксов из серии, когда на нём есть мужчина, женщина, то есть два человека, и они что-то обсуждают. А четвёртая тема весьма специфична. 

In [6]:
# выводим топы слов
for t, top_words in ldamodel.print_topics(num_topics=10, num_words=10):
    print("Topic", t, ":", top_words)

Topic 0 : 0.001*"b'day'" + 0.001*"b'planet'" + 0.001*"b'dot'" + 0.001*"b'beef'" + 0.001*"b'human'" + 0.001*"b'gliese'" + 0.001*"b'scientist'" + 0.001*"b'degree'" + 0.001*"b'easy'" + 0.001*"b'leopard'"
Topic 1 : 0.006*"b'woman'" + 0.005*"b'man'" + 0.002*"b'exhibit'" + 0.002*"b'female'" + 0.002*"b'goggles'" + 0.002*"b'child'" + 0.001*"b'found'" + 0.001*"b'male'" + 0.001*"b'link'" + 0.001*"b'text'"
Topic 2 : 0.022*"b'man'" + 0.012*"b'text'" + 0.012*"b'person'" + 0.011*"b'title'" + 0.009*"b'guy'" + 0.008*"b'woman'" + 0.006*"b'one'" + 0.006*"b'girl'" + 0.005*"b'just'" + 0.005*"b'hat'"
Topic 3 : 0.001*"b'nathan'" + 0.001*"b'title'" + 0.001*"b'text'" + 0.001*"b'roberts'" + 0.001*"b'turtle'" + 0.001*"b'cop'" + 0.001*"b'elaine'" + 0.001*"b'hatboy'" + 0.001*"b'cat'" + 0.001*"b'human'"
Topic 4 : 0.004*"b'line'" + 0.003*"b'labeled'" + 0.002*"b'wait'" + 0.002*"b'map'" + 0.002*"b'one'" + 0.002*"b'boy'" + 0.002*"b'graph'" + 0.001*"b'within'" + 0.001*"b'peter'" + 0.001*"b'island'"


In [7]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду
perplexity = ldamodel.log_perplexity(list(data))
print(2**(-perplexity))

351.2259898670064


давайте посчитаем величину перплексии на нашей модели. Перплексия — это очень распространённая мера качества тематических моделей, и особенное значение она имеет в контексте байесовских моделей. В первую очередь, потому что она описывает, насколько хорошо ваше распределение, описывает ваши исходные данные. И интерпретировать его можно так: чем меньше значение перплексии, тем лучше. С другой стороны, у перплексии нет максимального значения. То есть непонятно, если нам вручили число, непонятно, что оно означает. И ещё очень важно, что величина перплексии зависит, во-первых, от данных, а во-вторых, от количества тем. Поэтому, во-первых, нельзя сравнивать перплексию модели, построенных на разных данных, а во-вторых, нельзя по перплексии выбирать количество тем.

In [8]:
perp = ldamodel.bound(data)
2**(-perp/float(87409))

351.22599973675716

Давайте также обсудим ещё две важные функции, которые предоставляет gensim. Во-первых, это функция update, которая позволяет дообучить модель на новых данных. То есть, если, например, вы бы считали ещё один другой corpus, например, из новых комиксов, в переменную data2 и указали бы её как первый аргумент функции update и также указали бы, сколько сделать проходов по коллекции, чтобы дообучить модель, то gensim обновит все параметры. Ещё одна важная функция — это get_document_topics. Он позволяет для каждого документа найти распределение над множеством тем. Давайте получим распределение для другого документа и увидим, что в нём наибольшую роль играет третья тема.

In [9]:
# Добавление в модель новых документов, содержащихся в новом корупсе data2
ldamodel.update(data2, passes=10)

NameError: name 'data2' is not defined

In [14]:
# Получение распределения тем для конкретного документа
doc = list(data)[0]
ldamodel.get_document_topics(doc)

[(0, 0.055764973),
 (1, 0.056286175),
 (2, 0.056032903),
 (3, 0.05806509),
 (4, 0.77385086)]

Эти люди не знают про тематические модели:

![comic2](http://imgs.xkcd.com/comics/the_problem_with_wikipedia.png) | ![comic3](http://imgs.xkcd.com/comics/mystery_news.png)